In [72]:
# Define variables to predict

X_value = 24.5  # Number of kills to predict for over/under

player = "Kenny"
game_mode = "HardPoint"  # 'HardPoint', 'Search_and_Destroy', 'Control'
player_team = "OpTic_Texas"
enemy_team = "Carolina_Royal_Ravens"
teammates = ["Shotzzy", "Dashy", "Pred"]
enemies = ["Gwinn", "TJHaLy", "Clayster", "FeLo"]


In [73]:
from sklearn.base import BaseEstimator, ClassifierMixin
import torch
import numpy as np

class PyTorchModelWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        # PyTorch models are already trained
        return self

    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32)
            predictions = self.model(X_tensor)
            return (predictions >= 0.5).float().numpy().ravel()

    def predict_proba(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32)
            predictions = self.model(X_tensor)
            # Return probabilities for both classes (1-p and p)
            probabilities = torch.cat((1 - predictions, predictions), dim=1)
            return probabilities.numpy()


In [74]:
# Data loading and preparation

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


# Load the data
df = pd.read_csv("preprocessed_player_stats_Trial3.csv")

df["Over_X_Kills"] = df["Kills"].apply(lambda x: 1 if x > X_value else 0)

features = [
    "Mode_Control",
    "Mode_HardPoint",
    "Mode_Overall",
    "Mode_Search_and_Destroy",
    "PlayerTeam_Atlanta_FaZe",
    "PlayerTeam_Boston_Breach",
    "PlayerTeam_Carolina_Royal_Ravens",
    "PlayerTeam_Las_Vegas_Legion",
    "PlayerTeam_Los_Angeles_Guerrillas",
    "PlayerTeam_Los_Angeles_Thieves",
    "PlayerTeam_Miami_Heretics",
    "PlayerTeam_New_York_Subliners",
    "PlayerTeam_OpTic_Texas",
    "PlayerTeam_Seattle_Surge",
    "PlayerTeam_Toronto_Ultra",
    "EnemyTeam_Atlanta_FaZe",
    "EnemyTeam_Boston_Breach",
    "EnemyTeam_Carolina_Royal_Ravens",
    "EnemyTeam_Las_Vegas_Legion",
    "EnemyTeam_Los_Angeles_Guerrillas",
    "EnemyTeam_Los_Angeles_Thieves",
    "EnemyTeam_Miami_Heretics",
    "EnemyTeam_New_York_Subliners",
    "EnemyTeam_OpTic_Texas",
    "EnemyTeam_Seattle_Surge",
    "EnemyTeam_Toronto_Ultra",
    "04_Player",
    "04_Teammate",
    "04_Enemy",
    "aBeZy_Player",
    "aBeZy_Teammate",
    "aBeZy_Enemy",
    "Abuzah_Player",
    "Abuzah_Teammate",
    "Abuzah_Enemy",
    "Afro_Player",
    "Afro_Teammate",
    "Afro_Enemy",
    "Arcitys_Player",
    "Arcitys_Teammate",
    "Arcitys_Enemy",
    "Asim_Player",
    "Asim_Teammate",
    "Asim_Enemy",
    "Assault_Player",
    "Assault_Teammate",
    "Assault_Enemy",
    "Attach_Player",
    "Attach_Teammate",
    "Attach_Enemy",
    "Beans_Player",
    "Beans_Teammate",
    "Beans_Enemy",
    "Breszy_Player",
    "Breszy_Teammate",
    "Breszy_Enemy",
    "Cellium_Player",
    "Cellium_Teammate",
    "Cellium_Enemy",
    "Clayster_Player",
    "Clayster_Teammate",
    "Clayster_Enemy",
    "CleanX_Player",
    "CleanX_Teammate",
    "CleanX_Enemy",
    "Dashy_Player",
    "Dashy_Teammate",
    "Dashy_Enemy",
    "Diamondcon_Player",
    "Diamondcon_Teammate",
    "Diamondcon_Enemy",
    "Drazah_Player",
    "Drazah_Teammate",
    "Drazah_Enemy",
    "Envoy_Player",
    "Envoy_Teammate",
    "Envoy_Enemy",
    "Estreal_Player",
    "Estreal_Teammate",
    "Estreal_Enemy",
    "Fame_Player",
    "Fame_Teammate",
    "Fame_Enemy",
    "FelonY_Player",
    "FelonY_Teammate",
    "FelonY_Enemy",
    "Flames_Player",
    "Flames_Teammate",
    "Flames_Enemy",
    "Ghosty_Player",
    "Ghosty_Teammate",
    "Ghosty_Enemy",
    "Gio_Player",
    "Gio_Teammate",
    "Gio_Enemy",
    "Gwinn_Player",
    "Gwinn_Teammate",
    "Gwinn_Enemy",
    "Huke_Player",
    "Huke_Teammate",
    "Huke_Enemy",
    "HyDra_Player",
    "HyDra_Teammate",
    "HyDra_Enemy",
    "Insight_Player",
    "Insight_Teammate",
    "Insight_Enemy",
    "JoeDeceives_Player",
    "JoeDeceives_Teammate",
    "JoeDeceives_Enemy",
    "Kenny_Player",
    "Kenny_Teammate",
    "Kenny_Enemy",
    "KiSMET_Player",
    "KiSMET_Teammate",
    "KiSMET_Enemy",
    "Kremp_Player",
    "Kremp_Teammate",
    "Kremp_Enemy",
    "Lucky_Player",
    "Lucky_Teammate",
    "Lucky_Enemy",
    "MettalZ_Player",
    "MettalZ_Teammate",
    "MettalZ_Enemy",
    "Nastie_Player",
    "Nastie_Teammate",
    "Nastie_Enemy",
    "Nero_Player",
    "Nero_Teammate",
    "Nero_Enemy",
    "Pentagrxm_Player",
    "Pentagrxm_Teammate",
    "Pentagrxm_Enemy",
    "Pred_Player",
    "Pred_Teammate",
    "Pred_Enemy",
    "Priestahh_Player",
    "Priestahh_Teammate",
    "Priestahh_Enemy",
    "Purj_Player",
    "Purj_Teammate",
    "Purj_Enemy",
    "ReeaL_Player",
    "ReeaL_Teammate",
    "ReeaL_Enemy",
    "Scrap_Player",
    "Scrap_Teammate",
    "Scrap_Enemy",
    "Shotzzy_Player",
    "Shotzzy_Teammate",
    "Shotzzy_Enemy",
    "Sib_Player",
    "Sib_Teammate",
    "Sib_Enemy",
    "Simp_Player",
    "Simp_Teammate",
    "Simp_Enemy",
    "Skyz_Player",
    "Skyz_Teammate",
    "Skyz_Enemy",
    "SlasheR_Player",
    "SlasheR_Teammate",
    "SlasheR_Enemy",
    "Snoopy_Player",
    "Snoopy_Teammate",
    "Snoopy_Enemy",
    "TJHaLy_Player",
    "TJHaLy_Teammate",
    "TJHaLy_Enemy",
    "Vikul_Player",
    "Vikul_Teammate",
    "Vikul_Enemy",
    "Seany_Player",
    "Seany_Teammate",
    "Seany_Enemy",
    "oJohnny_Player",
    "oJohnny_Teammate",
    "oJohnny_Enemy",
]


X = df[features]
y = df["Over_X_Kills"]

# Remove constant features
X = X.loc[:, (X != X.iloc[0]).any()]

# Feature Selection
selector = SelectKBest(score_func=f_classif, k=20)
X_new = selector.fit_transform(X, y)

# Save the selected feature names
selected_features = X.columns[selector.get_support()].tolist()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [75]:
# Model building and training
import torch.nn as nn
import torch.optim as optim

class NeuralNetwork1(nn.Module):
    def __init__(self):
        super(NeuralNetwork1, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x

class NeuralNetwork2(nn.Module):
    def __init__(self):
        super(NeuralNetwork2, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x

class NeuralNetwork3(nn.Module):
    def __init__(self):
        super(NeuralNetwork3, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x

def train_model(model, X_train, y_train, num_epochs=20):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Train the models
model1 = NeuralNetwork1()
model2 = NeuralNetwork2()
model3 = NeuralNetwork3()

train_model(model1, X_train, y_train)
train_model(model2, X_train, y_train)
train_model(model3, X_train, y_train)


Epoch [2/20], Loss: 0.6892
Epoch [4/20], Loss: 0.6700
Epoch [6/20], Loss: 0.6498
Epoch [8/20], Loss: 0.6323
Epoch [10/20], Loss: 0.6166
Epoch [12/20], Loss: 0.6013
Epoch [14/20], Loss: 0.5841
Epoch [16/20], Loss: 0.5701
Epoch [18/20], Loss: 0.5542
Epoch [20/20], Loss: 0.5443
Epoch [2/20], Loss: 0.7123
Epoch [4/20], Loss: 0.6989
Epoch [6/20], Loss: 0.6911
Epoch [8/20], Loss: 0.6827
Epoch [10/20], Loss: 0.6704
Epoch [12/20], Loss: 0.6604
Epoch [14/20], Loss: 0.6519
Epoch [16/20], Loss: 0.6438
Epoch [18/20], Loss: 0.6330
Epoch [20/20], Loss: 0.6211
Epoch [2/20], Loss: 0.6539
Epoch [4/20], Loss: 0.6096
Epoch [6/20], Loss: 0.5749
Epoch [8/20], Loss: 0.5464
Epoch [10/20], Loss: 0.5135
Epoch [12/20], Loss: 0.4795
Epoch [14/20], Loss: 0.4531
Epoch [16/20], Loss: 0.4352
Epoch [18/20], Loss: 0.4104
Epoch [20/20], Loss: 0.3848


In [76]:
# Model evaluation
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the function to convert PyTorch model predictions to numpy
def model_predict(model, X):
    model.eval()
    with torch.no_grad():
        predictions = model(torch.tensor(X, dtype=torch.float32))
        return (predictions >= 0.5).float().numpy()

# Updated PyTorchModelWrapper class with predict_proba
class PyTorchModelWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        # PyTorch models are already trained
        return self

    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32)
            predictions = self.model(X_tensor)
            return (predictions >= 0.5).float().numpy().ravel()

    def predict_proba(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32)
            predictions = self.model(X_tensor)
            # Return probabilities for both classes (1-p and p)
            probabilities = torch.cat((1 - predictions, predictions), dim=1)
            return probabilities.numpy()
        
dt_model = DecisionTreeClassifier(random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
knn_model = KNeighborsClassifier(n_neighbors=5)
lr_model = LogisticRegression(solver='liblinear')

# Create voting classifier with additional models
ensemble = VotingClassifier(
    estimators=[
        ('nn1', PyTorchModelWrapper(model1)),
        ('nn2', PyTorchModelWrapper(model2)),
        ('nn3', PyTorchModelWrapper(model3)),
        ('dt', dt_model),
        ('rf', rf_model),
        ('knn', knn_model),
        ('lr', lr_model)
    ], voting='soft'
)

# Train the ensemble
ensemble.fit(X_train.numpy(), y_train.numpy().ravel())

# Evaluate the ensemble
ensemble_predictions = ensemble.predict(X_test.numpy())
accuracy = accuracy_score(y_test.numpy(), ensemble_predictions)
report = classification_report(y_test.numpy(), ensemble_predictions, zero_division=0)

print(f"Ensemble Test Accuracy: {accuracy}")
print(f"Ensemble Classification Report:\n{report}")

Ensemble Test Accuracy: 0.8717660292463442
Ensemble Classification Report:
              precision    recall  f1-score   support

         0.0       0.84      1.00      0.91       605
         1.0       0.99      0.61      0.75       284

    accuracy                           0.87       889
   macro avg       0.92      0.80      0.83       889
weighted avg       0.89      0.87      0.86       889



In [77]:
# Making predictions
def prepare_input(game_mode, player_team, enemy_team, player, teammates, enemies):
    input_data = {feature: 0 for feature in features}

    # Set game mode
    input_data[f"Mode_{game_mode}"] = 1

    # Set player team and enemy team
    input_data[f"PlayerTeam_{player_team}"] = 1
    input_data[f"EnemyTeam_{enemy_team}"] = 1

    # Set player
    input_data[f"{player}_Player"] = 1

    # Set teammates
    for teammate in teammates:
        input_data[f"{teammate}_Teammate"] = 1

    # Set enemies
    for enemy in enemies:
        input_data[f"{enemy}_Enemy"] = 1

    # Convert to DataFrame for prediction with correct feature names
    input_df = pd.DataFrame([input_data])

    # Remove constant features
    input_df = input_df.loc[:, input_df.columns.isin(selected_features)]

    # Select the same features as during training
    input_df = input_df[selected_features]

    # Standardize the data
    input_array = scaler.transform(input_df.to_numpy())

    return input_array


input_data = prepare_input(game_mode, player_team, enemy_team, player, teammates, enemies)

# Making a prediction with the ensemble
ensemble_prediction = ensemble.predict(input_data)
result = "Over" if ensemble_prediction[0] == 1 else "Under"
print(f'Ensemble Prediction: {result} {X_value} kills')


Ensemble Prediction: Under 25.0 kills
